# IBMi Jupyter Notebook Extension

In [ ]:
import pyodbc, getpass
import pandas as pd
from IPython.display import display, HTML, Image, Javascript
from IPython.core.magic import register_cell_magic, needs_local_scope

global _config
global _info

_config = {
    'host': '',
    'user': '',
    'pwd': ''
}

_info = {
    'sqlcode': 0,
    'sqlstate': '0',
    'sqlerror': ''
}

# disable pandas df display limit, trust user to limit result set in query
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# TODO: port? protocol? driver?
_config['host'] = input('Enter host: ').strip()
_config['user'] = input('Enter user: ').strip()
_config['pwd'] = getpass.getpass('Enter password: ')

## Define Utilities

In [ ]:
# Splitting cell content into SQL statements 
#  delimited by ';' (when not in string)
def get_statements(sql):
    stmts = []
    i = 0
    stmt = ''
    q = ''
    for c in sql:
        if c in ('\'', '"'):
            stmt += c
            q = '' if c == q else (c if q == '' else q)
        elif c == ';':
            stmts.append(stmt)
            stmt = ''
        else:
            stmt += c
    if stmt != '':
        stmts.append(stmt)
    return stmts

## Establish Connection

In [ ]:
print("Connecting to {} as {} ...".format(_config['host'], _config['user']))
try:
    conn = pyodbc.connect(driver='{IBM i Access ODBC Driver}', 
      system=_config['host'], uid=_config['user'], pwd=_config['pwd'])
    print('Successfully connected!')
except pyodbc.InterfaceError as e:
    print('Could not connect :(\n{}'.format(e))

## Setup Cell Magic

In [ ]:
# invoke with %%ibmi
@needs_local_scope 
@register_cell_magic
def ibmi(line, cell):
    if conn is None:
        print('Not connected.')
        return None
    i = 0
    stmts = get_statements(cell.strip())
    for stmt in stmts:
        sql = ' '.join([x.replace('{','{{').replace('}','}}').replace('\n','').strip() for x in stmt.split('\n')])
        i += 1
        print('Executing statement {} of {} ...'.format(i, len(stmts)))
        sql = sql.format(**globals())
        if line.strip() != '-':
            try:
                display(pd.read_sql(sql, conn))
            except TypeError:
                print('Statement executed successfully')
        else:
            conn.execute(sql)
del ibmi